In [1]:
import json
from pymongo import MongoClient

In [3]:
data_loc = "../result/"
# previously inserted 
# "results_cartimdbng20"#"results_server_imdbk3mid"

# second
# "resultsquora", 
data_name = "results quorak1 75v min 5 epochs100"
client = MongoClient('localhost', 27017)
db = client['thesis']
result = db.imdb # imdb

# if large database then improt via 
# mongoimport --db dbName --collection collectionName <fileName.json

### storing in mongodb

In [ ]:
with open("imdb.json", 'r') as result_file:
    result_json = json.load(result_file)
    for document in result_json["results"]:
#         Insert by remove other attribute except branchsize and algorithm and dataset
        #remove predict and true value beacuase it creates DocumentTooLarge Error since one document is above 16MB
        del document["train_true_predict"]
        del document["test_true_predict"]
        del document["filename"]
        del document["k_fold"]
        del document["d2v_vec_size"]
        del document["epochs"]
        del document["min_leaf_point"]
        del document["feature_size"]
        del document["d2v_shape"]
        del document["run"]
        del document["accuracy"]
        del document["precision"]
        del document["accuracy"]
        del document["precision"]
        del document["recall"]
        del document["f1"]
        del document["confusion_matrix"]
        del document["inner_node"]
        del document["leaf_node"]
        del document["all_node"]
        del document["training_time"]
        del document["tree_location"]
        del document["intermediate_result"]
        result.insert_one(document)

### Reading json for validation and checking

In [5]:
with open(data_loc+data_name+".json") as json_file: 
    data = json.load(json_file)
    
type(data["results"])
len(data["results"])

4

In [29]:
for i in data["results"]:
    print(i["dataset"],i["algorithm"], i["k_fold"],i["d2v_vec_size"], i["min_leaf_point"],i["epochs"], i["feature_size"])

quora lr_mvdt 3 25 5 100 2
quora lr_mvdt 3 25 5 100 5
quora lr_mvdt 3 25 5 100 10


In [ ]:
data["results"]

### Accuracy vs depth

In [2]:
dataset = 'quora'
vector_sizes_list = [10, 25, 50, 75, 100]
algorithms_list = ['lr_mvdt', 'rs_mvdt']
k = 5
epochs_list = [100, 300, 500, 800, 1000]
n_features_list = [2, 5, 10, 20, all]
min_leaf_point_list = [5, 10, 15, 20, 30]

In [3]:
by_list = [['by_n_features', '$feature_size'],
           ['by_min_leaf_points', '$min_leaf_point'],
           ['by_epochs', '$epochs'],
           ['by_vectors', '$d2v_vec_size'],
           ['by_k_folds', '$k_fold'],
           ['by_datasets', '$algorithm']]

In [9]:
for vector_size in vector_sizes_list:
    for algorithm in algorithms_list:    
        calculate(vector_size, algorithm)
        print(vector_size, algorithm)

10 lr_mvdt
10 rs_mvdt
25 lr_mvdt
25 rs_mvdt
50 lr_mvdt
50 rs_mvdt
75 lr_mvdt
75 rs_mvdt
100 lr_mvdt
100 rs_mvdt


In [6]:
def calculate(vector_size, algorithm):    
    query = [{'$match': {'dataset': dataset, 'd2v_vec_size': vector_size, 'algorithm': algorithm}},
         {'$project':{'intermediate_result': 1}},        
         ]
    results = db['quora'].aggregate(query)

    for result in results:        
        for i in result['intermediate_result']:
            if i is not None:
                new_result = {'dataset': dataset,
                              'd2v_vec_size': vector_size,
                              'algorithm': algorithm,                          
                              'on_depth': i['on_depth'],
                              'train_acc': i['accuracy'][0],
                              'test_acc': i['accuracy'][1]
                             }
                
                db.acc_vs_depth_quora.insert_one(new_result)

In [9]:
###temp insert

In [5]:
query = [{'$match': {'k_fold':3}}]
results = db['temp_imdb'].aggregate(query)

for result in results:
    db.imdb.insert_one(result)